In [11]:
import pandas as pd
import numpy as np
from pandas  import Series
import os
import compare
import sys
import argparse
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from tensorflow.python.platform import gfile
from scipy import misc
import tensorflow as tf
import facenet
import align.detect_face

class ImageClass():
    "Stores the paths to images for a given class"
    def __init__(self, name, image_paths):
        self.name = name
        self.image_paths = image_paths
  
    def __str__(self):
        return self.name + ', ' + str(len(self.image_paths)) + ' images'
  
    def __len__(self):
        return len(self.image_paths)
    
    

def get_dataset(path, has_class_directories=True):
    dataset = []
    path_exp = os.path.expanduser(path)
    print('path_exp')
    print(path_exp)
    classes = os.listdir(path_exp)
    print(classes)
    classes.sort()
    nrof_classes = len(classes)
    for i in range(nrof_classes):
        class_name = classes[i]
        print(class_name)
        facedir = os.path.join(path_exp, class_name)
        image_paths = get_image_paths(facedir)
        print(image_paths)
        dataset.append(ImageClass(class_name, image_paths))
    
    return dataset

def get_image_paths(facedir):
    image_paths = []
    if os.path.isdir(facedir):
        images = os.listdir(facedir)
        image_paths = [os.path.join(facedir,img) for img in images]
    return image_paths

    return images

def parse_arguments(argv):
    parser = argparse.ArgumentParser()
   
    parser.add_argument('model', type=str, 
        help='Could be either a directory containing the meta_file and ckpt_file or a model protobuf (.pb) file')
    parser.add_argument('image_files', type=str, nargs='+', help='Images to compare',default='')
    parser.add_argument('--image_size', type=int,
        help='Image size (height, width) in pixels.', default=160)
    parser.add_argument('--margin', type=int,
        help='Margin for the crop around the bounding box (height, width) in pixels.', default=44)
    parser.add_argument('--gpu_memory_fraction', type=float,
        help='Upper bound on the amount of GPU memory that will b e used by wthe process.', default=1.0)
    return parser.parse_args(argv)

def load_model(model):
    # Check if the model is a model directory (containing a metagraph and a checkpoint file)
    #  or if it is a protobuf file with a frozen graph
    model_exp = os.path.expanduser(model)
    print('')
    if (os.path.isfile(model_exp)):
        print('Model filename: %s' % model_exp)
        with gfile.FastGFile(model_exp,'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
    else:
        print('Model directory: %s' % model_exp)
        meta_file, ckpt_file = get_model_filenames(model_exp)
        
        print('Metagraph file: %s' % meta_file)
        print('Checkpoint file: %s' % ckpt_file)
      
        saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file))
        saver.restore(tf.get_default_session(), os.path.join(model_exp, ckpt_file))
        
        
def load_and_align_data(image_paths, image_size, margin, gpu_memory_fraction):

    minsize = 20 # minimum size of face  
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    
    print('Creating networks and loading parameters')
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
  
    nrof_samples = len(image_paths)
    print(nrof_samples)
    labels=[]
    img_list = [None] * nrof_samples
    for i in range(nrof_samples):
        img = misc.imread(os.path.expanduser(image_paths[i]), mode='RGB')
        labels.append(os.path.expanduser(image_paths[i]))
        img_size = np.asarray(img.shape)[0:2]
        bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
        print('bounding_boxes')
        print(bounding_boxes)
        det = np.squeeze(bounding_boxes[0,0:4])
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2, img_size[1])
        bb[3] = np.minimum(det[3]+margin/2, img_size[0])
        cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
        aligned = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
        prewhitened = facenet.prewhiten(aligned)
        img_list[i] = prewhitened
    images = np.stack(img_list) 
    return images,labels

def my_get_paths(data_dir):
    image_patsh=[]
    for img_path in os.listdir(data_dir):
        print(img_path)
        image_patsh.append(img_path)
    return image_patsh;

image_parth='images'
dataset=my_get_paths(image_parth)
images,labels=load_and_align_data(dataset,160,44,1.0)
print(images.shape)
print(dataset)

with tf.Graph().as_default():

        with tf.Session() as sess:
      
            # Load the model
            facenet.load_model('20170511-185253.pb')
    
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

            # Run forward pass to calculate embeddings
            feed_dict = { images_placeholder: images, phase_train_placeholder:False }
            emb = sess.run(embeddings, feed_dict=feed_dict)
            
print(labels)
print(emb)


image1.jpg
image2.jpg
image3.jpg
image4.jpg
image5.jpg
Creating networks and loading parameters
5
bounding_boxes
[[  88.31653426   24.61487465  199.91473259  164.09535029    0.99999833]]
bounding_boxes
[[ 110.54291509  106.92719141  386.21587306  430.86049597    0.99982893]]
bounding_boxes
[[ 277.64438201  154.84847774  629.04146487  577.49820559    0.99979287]]
bounding_boxes
[[  56.33432162   56.32604555  115.2576192   133.37218186    0.99981827]]
bounding_boxes
[[ 209.46522573   49.83780602  305.50001413  179.01723082    0.99956745]]
(5, 160, 160, 3)
['image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg']
Model filename: 20170511-185253.pb
['image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg']
[[  3.82261612e-02   1.32316258e-03  -8.31084475e-02   8.07684213e-02
   -4.16699722e-02  -5.22459783e-02   7.79244825e-02   1.13583319e-01
    5.76144382e-02   1.19759686e-01   8.36387370e-03   4.95191328e-02
    6.68582395e-02  -1.08106665e-01   8.88785422e-02 

In [ ]:
import pandas as pd
import numpy as np
from pandas  import Series
import os

data1=pd.DataFrame(np.random.randn(1,5))
data2=pd.DataFrame(['test'],columns=["keyword"])
res=pd.concat([data1,data2],axis=1, ignore_index=True)    #合并两个DF
res.to_csv('data1.csv',index=False)  #保存进CSV

data_info=pd.read_csv('data1.csv',header=None)   #读出ＣＳＶ数据到data_info
data3=pd.DataFrame(np.random.randn(1,5))          
data4=pd.DataFrame(['test2'],columns=["keyword"]) 

res2=pd.concat([data3,data4],axis=1, ignore_index=True)     #合并两个DF
data_info=data_info.append(res2)  

print(data_info)
data_info.to_csv('data1.csv',index=False,header=None)


data_info2=pd.read_csv('data1.csv')
print(data_info2)
columns= data_info2.iloc[0:,0:6]
print('col')
print(columns)
rows=columns.shape[0]
for i in range(rows):
   


         0         1         2         3         4      5
0  0.00000  1.000000  2.000000  3.000000  4.000000      5
1  0.12951 -0.297169  0.562703  0.581894  0.560206   test
0  1.47759 -0.211290  0.611100 -0.287762 -0.573065  test2
       0.0       1.0       2.0       3.0       4.0      5
0  0.12951 -0.297169  0.562703  0.581894  0.560206   test
1  1.47759 -0.211290  0.611100 -0.287762 -0.573065  test2
col
       0.0       1.0       2.0       3.0       4.0      5
0  0.12951 -0.297169  0.562703  0.581894  0.560206   test
1  1.47759 -0.211290  0.611100 -0.287762 -0.573065  test2
